In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
import json
import random
from keras.models import load_model

In [ ]:
model1 = load_model('/content/gdrive/MyDrive/project/final/models/chatbot_model.h5')

intents = json.loads(open('/content/gdrive/MyDrive/project/final/chatbot/intents.json').read())
words = pickle.load(open('/content/gdrive/MyDrive/project/final/chatbot/words.pkl','rb'))
classes = pickle.load(open('/content/gdrive/MyDrive/project/final/chatbot/classes.pkl','rb'))

In [ ]:
from keras.preprocessing.sequence import pad_sequences
model2 = load_model('/content/gdrive/MyDrive/project/final/models/emotion_model.h5')

with open('/content/gdrive/MyDrive/project/final/emotion detection/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [ ]:
import urllib.request
import re
import random

rmsystem={
    "anger" : ["control your temper"],
    "sadness" : ["stop being depressed"],
    "anxious" : ["reduce anxiety"],
    "suicide" : ["stop suicidal thoughts"],
}

def recommend(ans):
  search = rmsystem[ans][0]
  search = search.replace(" ","+")
  html = urllib.request.urlopen("https://www.youtube.com/results?search_query=" + search)
  video_ids = re.findall(r"watch\?v=(\S{11})", html.read().decode())
  i=random.randint(1, len(video_ids))
  str = """"""
  if(ans == "suicide"):
    str = "Please call helpline: 9152987821.<br>"
  str += "<a href = \"https://www.youtube.com/watch?v=" + video_ids[i] +"\" target=\"_blank\">This might help you</a>"
  return str

def get_key(value):
    dictionary={'joy':0,'anger':1,'suicide':2,'sadness':3,'anxious':4}
    for key,val in dictionary.items():
          if (val==value):
            return key

def predict(sentence):
  sentence_lst=[]
  sentence_lst.append(sentence)
  sentence_seq=tokenizer.texts_to_sequences(sentence_lst)
  sentence_padded=pad_sequences(sentence_seq,maxlen=80,padding='post')
  value = model2.predict(sentence_padded)
  ans = get_key(np.argmax(value))
  value = np.amax(value)
  if (ans != "joy" and value > 0.8):
    return ans
  return 0

In [ ]:
import requests

def NewsFromBBC():
  query_params = {"source": "bbc-news","sortBy": "top","apiKey": "68a3de083c3741539c6135322a75110f"}
  main_url = "https://newsapi.org/v1/articles"
  res = requests.get(main_url, params=query_params)
  open_bbc_page = res.json()
  article = open_bbc_page["articles"]
  res = ""
  for ar in article:
    res += "-><a href="+ar["url"]+" target=\"_blank\">"+ar["title"]+"</a><br>"
  return res

In [ ]:
def predict_class(sentence, model1):
    p = bow(sentence, words,show_details=False)
    res = model1.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== ints):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model1)
    ints = ints[0]['intent']
    if(ints=="agent.news"):
      return NewsFromBBC(), ints
    res = getResponse(ints, intents)
    return res, ints


In [ ]:
!pip install flask-ngrok

In [ ]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/get")
def chat():
  major = ["agent.story", "recommend.songs", "agent.news", "greetings.bye"]
  userText = request.args.get('msg')
  response,intent = chatbot_response(userText)
  if(intent in major):
    return response
  ans = predict(userText)
  if(ans == 0):
    return response
  else:
    return recommend(ans)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://badb73d0e9c7.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/Jun/2021 07:20:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2021 07:20:24] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2021 07:20:25] "GET /favicon.ico HTTP/1.1" 404 -
